# 챗봇 만들기

### https://jfun.tistory.com/199?category=851249
### https://www.slideshare.net/KimSungdong1/20170227-72644192

In [ ]:
input_data = '판교에 지금 주문해줘'
output_data = ''

request = {
    "intent_id":"",
    "input_data":input_data,
    "request_type":"text",
    "story_slot_entity":{},
    "output_data":output_data
}

## 기본 데이터 셋(DB))

In [ ]:
intent_list = {
    "주문":["주문", "배달"],
    "예약":["예약", "잡아줘"],
    "정보":["정보","알려"]
}

story_slot_entity = {"주문":{"메뉴":None, "장소":None, "날짜":None},
"예약":{"장소": None, "날짜":None},
"정보":{"대상":None}
}

## 형태소 분석

In [ ]:
from eunjeon import Mecab
mecab = Mecab()
preprocessed = mecab.pos(request.get('input_data'))

print(preprocessed)

## Intent 도출(Ruled Based)

In [ ]:
# Char CNN을 사용해서 연결하면 좀 더 쉽게 만들 수 있다
intent_id = "주문"
slot_value = story_slot_entity.get("주문")

## NER 도출(Ruled based)

In [ ]:
# LSTM 기법을 사용해서 연결하면 좀 더 쉬울 수 있다
menu_list = ["피자", "햄버거", "치킨"]
loc_list = ["판교", "야탑", "서현"]
date_list = ["지금", '내일', '모레']

## Dictionary 기반 slot 구성

In [ ]:
for pos_tag in preprocessed:
    if pos_tag[1] in ['NNG', 'NNP', 'SL', 'MAG']:
        if pos_tag[0] in menu_list:
            slot_value['메뉴'] = pos_tag[0]
        elif pos_tag[0] in loc_list:
            slot_value['장소'] = pos_tag[0]
        elif pos_tag[0] in date_list:
            slot_value['날짜'] = pos_tag[0]


print(story_slot_entity.get('주문'))

## 빈 slot 검색

In [ ]:
if None in slot_value.values():
    key_values=""
    for key in slot_value.keys():
        if slot_value[key] is None:
            key_values = key_values + key + ""
    output_data = key_values + "선택해주세요"
else:
    output_data="주문이 완료 되었습니다."

print(output_data)

In [ ]:
response = {
    "intent_id":"",
    "input_data":input_data,
    "request_type":"text",
    "story_slot_entity":{},
    "output_data":""
}

response["output_data"]=output_data

print(response["output_data"])

# Word Representation

In [14]:
ona_data = [
    ['안녕', '만나서 반가워'],
    ['넌 누구니','나는 AI봇이란다.'],
    ['피자 주문할께', '음료도 주문해줘'],
    ['음료는 뭘로', '콜라로 해줘']
]

train_gata = list(map(lambda x: mecab.morphs(''.join(x)), ona_data))

import itertools

train_data = list(itertools.chain.from_iterable(train_gata))

print(list(train_data))

['안녕', '만나', '서', '반가워', '넌', '누구', '니', '나', '는', 'AI', '봇', '이', '란다', '.', '피자', '주문', '할께', '음료', '도', '주문', '해', '줘', '음료', '는', '뭘로', '콜라', '로', '해', '줘']


In [16]:
import numpy as np

bucket = np.zeros(len(train_data), dtype=np.float)

for word in train_data:
    bucket_temp = bucket.copy()
    np.out(bucket_temp, train_data.index(word), 1)
    print(bucket_temp)

AttributeError: module 'numpy' has no attribute 'out'